In [ ]:
%matplotlib inline

# Librosa Demo

In this demo, I hope to show you guys some of the things that you can do with the `librosa` library.

We will explore the following:
 - How we can get information about the beat events from an audio clip
 - How to convert beat events into time stamps
 - How to obtain the chromagram and apply some enhancements to it

A canonical example can be found at [http://librosa.github.io/librosa/](http://librosa.github.io/librosa/).

Credits to Brian McFee for the code on chromagram enhancements.

In [ ]:
from __future__ import print_function
import librosa

We can set the name of the file of our audio example

In [ ]:
filename = "titlescreen.wav"

Load the audio as waveform `y`, and store the sampling rate as `sr`.

In [ ]:
y, sr = librosa.load(filename)

Let's run the default beat tracker.

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

Then we can convert the frame indices of beat events into timestamps, and output it into a csv file.

In [ ]:
beat_times = librosa.frames_to_time(beat_frames, sr=sr)
librosa.output.times_csv('beat_times.csv', beat_times)

For the next demo, we need to import a little bit more stuff.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

import librosa.display

First, let's get the chroma feature, using the constant-Q transformation (CQT).

In [ ]:
chroma_orig = librosa.feature.chroma_cqt(y=y, sr=sr)

For this demo, let's zoom in on a 20-second chunk from the middle of the song.

In [ ]:
idx = [slice(None), slice(*list(librosa.time_to_frames([0, 20])))]

And for comparison, we'll show the CQT matrix as well.

In [ ]:
C = np.abs(librosa.cqt(y=y, sr=sr, bins_per_octave=12*3, n_bins=7*12*3))

Now, we can plot the original chroma.

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max)[idx],
                         y_axis='cqt_note', bins_per_octave=12*3)
plt.colorbar()

plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_orig[idx], y_axis='chroma')
plt.colorbar()
plt.ylabel('Original')
plt.tight_layout()

To further improve our feature chroma, we can try over-sampling the frequency axis to reduce sensitivity to tuning deviations.

In [ ]:
chroma_os = librosa.feature.chroma_cqt(y=y, sr=sr, bins_per_octave=12*3)

Let's plot it again, to see the difference between our original chroma, and the new chroma that we have oversampled.

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(2, 1, 1)
librosa.display.specshow(chroma_orig[idx], y_axis='chroma')
plt.colorbar()
plt.ylabel('Original')

plt.subplot(2, 1, 2)
librosa.display.specshow(chroma_os[idx], y_axis='chroma', x_axis='time')
plt.colorbar()
plt.ylabel('3x-over')
plt.tight_layout()